In [48]:
%matplotlib inline
import pandas as pd
import numpy as np
predictionFeature = 'SP.ADO.TFRT'
X = pd.read_csv("data/xSave.csv", index_col=0)
y = pd.read_csv("data/ySave.csv", index_col=0) #, usecols=['YEAR_5'])
columns=['CountryCode', 'IndicatorCode', 'Year', 'Value']
data = pd.read_csv("data/indicators.csv", usecols=(columns))
X.head()

,COUNTRY,YEAR,NY.GDP.MKTP.CD_0,NY.GDP.PCAP.CD_0,AG.PRD.CREL.MT_0,AG.YLD.CREL.KG_0,AG.PRD.CROP.XD_0,AG.PRD.FOOD.XD_0,AG.LND.CREL.HA_0,AG.PRD.LVSK.XD_0,...,SP.DYN.TFRT.IN_4,TX.VAL.MRCH.CD.WT_4,TX.VAL.MRCH.WL.CD_4,TX.VAL.MRCH.RS.ZS_4,TX.VAL.MRCH.HI.ZS_4,TM.VAL.MRCH.CD.WT_4,TM.VAL.MRCH.WL.CD_4,TM.VAL.MRCH.RS.ZS_4,TM.VAL.MRCH.HI.ZS_4,TG.VAL.TOTL.GD.ZS_4
0,AFG,1971,1.673333e+09,161.384702,3826000.0,1224.516,83.05,70.20,3124500.0,60.38,...,7.450,100000000.0,99900000.0,47.314314,42.712713,141000000.0,145386000.0,32.425405,57.253106,13.161423
1,AFG,1977,1.733333e+09,144.173944,4270000.0,1279.592,91.25,73.56,3337000.0,58.45,...,7.449,306000000.0,313137000.0,45.115397,35.590492,328000000.0,458652000.0,45.553055,45.343964,21.467268
2,AFG,1978,2.155555e+09,175.027098,4351000.0,1301.915,94.92,76.02,3342000.0,61.00,...,7.449,321000000.0,269156000.0,43.718141,27.069432,395000000.0,450516000.0,53.467801,34.542835,21.696969
3,AFG,1979,2.366667e+09,188.085136,4481000.0,1316.392,97.68,78.24,3404000.0,63.38,...,7.449,474000000.0,435015000.0,60.016551,18.152248,425000000.0,551216000.0,68.909647,24.561515,24.310830
4,AFG,1980,2.555556e+09,199.164809,4624000.0,1362.404,102.93,83.72,3394000.0,69.07,...,7.449,670000000.0,670474000.0,64.592065,19.471896,841000000.0,494754000.0,60.669343,32.162044,41.491344


In [49]:
y.head()

,YEAR_0,YEAR_1,YEAR_2,YEAR_3,YEAR_4
0,145.3210,146.2150,147.1090,148.0030,148.8970
1,150.8782,151.9654,153.0526,154.1398,155.2270
2,151.9654,153.0526,154.1398,155.2270,156.3758
3,153.0526,154.1398,155.2270,156.3758,157.5246
4,154.1398,155.2270,156.3758,157.5246,158.6734


In [50]:
# Can use OBSERVATION to get new y Data if we want to train on something else
pData = data[data.IndicatorCode == predictionFeature]
pData.head()

# If I do this I need to add some value to year and filter for things that dont have
# it (eg dont have 'SP.ADO.TFRT' 4 years from now, drop this observation)
# Also need to remove filtering code from other file
indices = [(pData.CountryCode == row[0]) & (pData.Year == int(row[1])) for row in X.values]
values = [pData[index].Value.values[0] for index in indices]
#y = pd.DataFrame(values, index=X.OBSERVATION.values, columns=['Value'])

In [51]:
from sklearn.cross_validation import train_test_split
X_data = X.drop(['COUNTRY', 'YEAR'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X_data, y, test_size=0.2, random_state=0)

In [52]:
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import cross_val_score
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [53]:
train = y_train.YEAR_4 
test = y_test.YEAR_4

In [54]:
from sklearn.linear_model import LinearRegression
clf = LinearRegression()
clf.fit(X_train, train)
#cv_scores = cross_val_score(clf, X_test, test, cv=4)
#cv_scores
clf.score(X_test, test)

0.99368182314774856

In [55]:
from sklearn.svm import SVR
clf = SVR()
clf.fit(X_train, train)
clf.score(X_test, test)

0.87752602923588452

In [56]:
#from sklearn.grid_search import GridSearchCV
#param_grid = [
#    {'C': [10], 'kernel': ['linear']},
#    {'C': [10], 'gamma': [0.03], 'kernel': ['rbf']},
#]
#clf = GridSearchCV(SVR(), param_grid, cv=4)
#clf = clf.fit(X_train, train)
#clf = clf.best_estimator_
#clf.score(X_test, test)

In [57]:
import seaborn as sb
import matplotlib.pyplot as plt
def getGraphData(clf, country):
    indices = (X.COUNTRY == country)
    realValues = y[indices].YEAR_4
    plotData = X[indices]
    
    dataPoints = []
    predictions = clf.predict(plotData.drop(['COUNTRY', 'YEAR'], axis=1))
    for i in range(len(plotData.YEAR)):
        dataPoints.append([country + "_PREDICTED", plotData.YEAR[i], predictions[i], True])
        dataPoints.append([country + "_REAL", plotData.YEAR[i], realValues[i], False])
    return len(plotData.YEAR), dataPoints

def testClf(clf, countries):
    countryGraphData = []
    for country in countries:
        numYears, dataPoints = getGraphData(clf, country)
        countryGraphData.append(dataPoints)
    
    countryGraphDataFlattened = [item for sublist in countryGraphData for item in sublist]
    
    graphData = pd.DataFrame(dataPoints, columns=['COUNTRY', 'YEAR', 'VALUE', 'IS_PREDICTED'])
    
    numYears = len(countries)
    
    linestyles = ["--", "-"]
    markers= ["x", "o"]
    palette = np.repeat(sb.color_palette("Set1", n_colors=numYears, desat=.5), 2, axis=0)

    graph = sb.pointplot(x='YEAR', y='VALUE', hue='COUNTRY', linestyles=linestyles, markers=markers, palette=palette, data=graphData)

In [59]:
#testClf(clf, ['ALB'])

In [60]:
country = 'ALB'
indices = (X.COUNTRY == country)
realValues = y[indices].YEAR_4
plotData = X[indices]
plotData

,COUNTRY,YEAR,NY.GDP.MKTP.CD_0,NY.GDP.PCAP.CD_0,AG.PRD.CREL.MT_0,AG.YLD.CREL.KG_0,AG.PRD.CROP.XD_0,AG.PRD.FOOD.XD_0,AG.LND.CREL.HA_0,AG.PRD.LVSK.XD_0,...,SP.DYN.TFRT.IN_4,TX.VAL.MRCH.CD.WT_4,TX.VAL.MRCH.WL.CD_4,TX.VAL.MRCH.RS.ZS_4,TX.VAL.MRCH.HI.ZS_4,TM.VAL.MRCH.CD.WT_4,TM.VAL.MRCH.WL.CD_4,TM.VAL.MRCH.RS.ZS_4,TM.VAL.MRCH.HI.ZS_4,TG.VAL.TOTL.GD.ZS_4
11,ALB,1994,2.101625e+09,639.463899,897000.0,2794.393,84.31,66.95,321000.0,55.48,...,2.772,1.427000e+08,1.414767e+08,12.001195,87.658176,6.070000e+08,1.605756e+08,1.116874,82.313646,37.755446
12,ALB,1995,1.139167e+09,348.711318,446152.0,2071.859,60.99,57.35,215339.0,54.01,...,2.723,2.020000e+08,2.018372e+08,16.279704,82.611252,7.135000e+08,6.506324e+08,1.973571,81.166435,37.760378
13,ALB,1996,7.094526e+08,218.492166,430122.0,2269.282,66.81,62.06,189541.0,60.59,...,2.670,2.112500e+08,2.105400e+08,8.953603,90.885497,9.367900e+08,9.085893e+08,3.388747,81.482433,34.632737
14,ALB,1997,1.228071e+09,380.527371,665720.0,2685.048,80.47,70.85,247936.0,65.38,...,2.611,1.390000e+08,1.413009e+08,4.011440,95.973605,6.461800e+08,6.197354e+08,0.191882,89.339587,33.271705
15,ALB,1998,1.985674e+09,619.065163,646026.0,2460.911,79.00,78.06,262515.0,77.09,...,2.543,2.077400e+08,2.064882e+08,2.807034,96.898791,8.417600e+08,7.946468e+08,0.259732,89.735113,38.768083
16,ALB,1999,2.424499e+09,760.559376,645400.0,2841.457,81.28,84.87,227137.0,88.31,...,2.467,3.511200e+08,2.753493e+08,3.173054,96.768918,1.154290e+09,8.991029e+08,0.123588,86.883471,44.085370
17,ALB,2000,3.314898e+09,1046.358511,503714.0,2450.853,80.35,84.53,205526.0,90.02,...,2.383,2.579300e+08,2.577226e+08,2.759107,95.662539,1.090000e+09,1.103512e+09,1.714447,84.259304,37.112161
18,ALB,2001,2.359903e+09,749.584649,602043.0,2833.304,82.78,79.46,212488.0,78.85,...,2.291,3.071900e+08,3.073212e+08,3.111091,93.294087,1.326920e+09,1.335514e+09,1.191981,82.961969,40.241494
19,ALB,2002,2.707124e+09,865.302162,605746.0,2872.114,88.78,83.21,210906.0,80.02,...,2.195,3.395500e+08,3.388436e+08,2.280825,94.766864,1.503720e+09,1.501551e+09,0.896003,82.379280,41.561157
20,ALB,2003,3.414761e+09,1098.425463,497500.0,2798.088,81.91,82.49,177800.0,84.47,...,2.097,4.479500e+08,4.471149e+08,2.357371,95.253119,1.864130e+09,1.852325e+09,0.489174,78.847546,40.231456


In [62]:
dataPoints = []
predictions = clf.predict(plotData.drop(['COUNTRY', 'YEAR'], axis=1))
predictions

array([ 55.3324835,  55.3324835,  55.3324835,  55.3324835,  55.3324835,
        55.3324835,  55.3324835,  55.3324835,  55.3324835,  55.3324835,
        55.3324835,  55.3324835,  55.3324835,  55.3324835,  55.3324835,
        55.3324835])